In [1]:
import mlflow
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

data = load_iris()
X, y = data.data, data.target
model = DecisionTreeClassifier()
model.fit(X, y)

y_pred = model.predict(X)
accuracy = accuracy_score(y, y_pred)
print(f"Model accuracy: {accuracy:.2f}")


Model accuracy: 1.00


In [2]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [3]:
mlflow.set_experiment("MLflow Iris Experiment")

<Experiment: artifact_location='mlflow-artifacts:/994234090907559238', creation_time=1746292595175, experiment_id='994234090907559238', last_update_time=1746292595175, lifecycle_stage='active', name='MLflow Iris Experiment', tags={}>

In [4]:
from mlflow.models import infer_signature
DT_params = {
    "criterion": "gini",
    "max_depth": 3,
    "min_samples_split": 2,
    "min_samples_leaf": 1,
}

model = DecisionTreeClassifier(criterion=DT_params["criterion"],
                               max_depth=DT_params["max_depth"],
                               min_samples_split=DT_params["min_samples_split"],
                               min_samples_leaf=DT_params["min_samples_leaf"])
model.fit(X, y)
y_pred = model.predict(X)
accuracy = accuracy_score(y, y_pred)

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(DT_params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic DT model for iris data")

    # Infer the model signature
    signature = infer_signature(X, model.predict(X))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="iris_model_DT",
        signature=signature,
        input_example=X,
        registered_model_name="DT_iris_model",
    )

Registered model 'DT_iris_model' already exists. Creating a new version of this model...
2025/05/09 15:04:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: DT_iris_model, version 2


🏃 View run amazing-bear-71 at: http://127.0.0.1:5000/#/experiments/994234090907559238/runs/aa9d14ba1871479baed24f40296e929d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/994234090907559238


Created version '2' of model 'DT_iris_model'.


In [5]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [6]:
predicted = loaded_model.predict(X)
accuracy = accuracy_score(y, predicted)
print(f"Loaded model accuracy: {accuracy:.2f}")


Loaded model accuracy: 0.97


In [7]:
from sklearn.ensemble import RandomForestClassifier

mlflow.autolog()

rf = RandomForestClassifier(n_estimators=100, max_depth=6, max_features=3)
# MLflow triggers logging automatically upon model fitting
rf.fit(X, y)
y_pred = rf.predict(X)
accuracy = accuracy_score(y, y_pred)
print(f"Random Forest Model accuracy: {accuracy:.2f}")

# The model is automatically logged to MLflow, so we don't need to log it manually
# You can view the logged model in the MLflow UI
# mlflow ui --backend-store-uri sqlite:///mlruns.db --host

2025/05/09 15:23:33 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/09 15:23:33 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e966d9ee449543e49e0810ecc33c8498', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


🏃 View run redolent-shad-638 at: http://127.0.0.1:5000/#/experiments/994234090907559238/runs/e966d9ee449543e49e0810ecc33c8498
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/994234090907559238
Random Forest Model accuracy: 1.00


In [9]:
from sklearn.linear_model import LogisticRegression
mlflow.autolog()

log_reg = LogisticRegression(max_iter=1000)

# MLflow triggers logging automatically upon model fitting
log_reg.fit(X, y)
y_pred = log_reg.predict(X)
accuracy = accuracy_score(y, y_pred)
print(f"Logistic Regression Model accuracy: {accuracy:.2f}")

2025/05/09 15:25:36 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/09 15:25:36 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '7da58041b8e3414e907ce5761b770427', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


🏃 View run marvelous-snail-800 at: http://127.0.0.1:5000/#/experiments/994234090907559238/runs/7da58041b8e3414e907ce5761b770427
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/994234090907559238
Logistic Regression Model accuracy: 0.97


In [10]:
from sklearn.linear_model import LogisticRegression
mlflow.autolog()

log_reg = LogisticRegression(max_iter=10)

# MLflow triggers logging automatically upon model fitting
log_reg.fit(X, y)
y_pred = log_reg.predict(X)
accuracy = accuracy_score(y, y_pred)
print(f"Logistic Regression Model accuracy: {accuracy:.2f}")

2025/05/09 15:29:00 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/09 15:29:01 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f40be9e450bb4abf94e3399c23cad5d5', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
c:\Users\engsm\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


🏃 View run capricious-whale-506 at: http://127.0.0.1:5000/#/experiments/994234090907559238/runs/f40be9e450bb4abf94e3399c23cad5d5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/994234090907559238
Logistic Regression Model accuracy: 0.97
